<img src='./IMG/S_2_fires.png' alt='Logo Head_' align='center' width='99%'></img>
<br>
<img src='./IMG/header_1.png' alt='Logo UNSPIDER' align='left' width='50%'></img>
<br>

# FOREST FIRE MONITORING ON QGIS-EARTHENGINE-PLUGIN INTEGRATES GOOGLE EARTH ENGINE AND QGIS USING PYTHON API

##	1. Integrates Google Earth Engine and QGIS using Python API for Fires Monitoring

<a href="https://colab.research.google.com/github/Alexanderariza/qgis-earthengine-Fires-Monitoring/blob/master/qgis-earthengine-Fires-Monitoring.ipynb"><img src='./IMG/COLAB.svg' alt='Logo CO' align='left' width='10%'></img>
<br>

<a href="https://nbviewer.jupyter.org/github/Alexanderariza/qgis-earthengine-Fires-Monitoring/blob/master/qgis-earthengine-Fires-Monitoring.ipynb"><img src='./IMG/NT_vie.svg' alt='NT_VW' align='left' width='10%'></img>
<br>

###	Installation
The plugin can be installed from the **QGIS** Plugin Repository as any other plugin. It may take some time install (~30-60 sec) due to EE dependencies included in the distribution.

<div class="alert alert-block alert-success" align="left">
<b><i>  "Warning: Update local installations to v0.1.231 or greater. Earlier versions are not compatible with recent changes to the Earth Engine backend.
Check version from Python:"</i></b>
</div>

In [21]:
import ee
print(ee.__version__)

0.1.226


Next we install the Google Earth Engine plugin for **QGIS**. To do this, we must go to the top menu of QGIS and click on (`Add-ons>`) Manage and install add-ons ... In the search box we type **"Google Earth"**, select the plugin and install it:

<img src='./IMG/qgis_gee_1.png' alt='GEE plugin' align='center' width='99%'></img>
<br>

The user needs to have an active **Google Earth Engine (EE)** account to use the plugin. If you don’t have one - please sign-up [**here:**](https://earthengine.google.com/signup/) https://earthengine.google.com/signup/

After the installation, the plugin checks if the user is authenticated to use the **GEE**. If this is not the case - the user will be asked to authenticate.
Once installed and authenticated, the plugin can be accessed from the (`*QGIS Python Code Editor*`) to write and execute EE scripts. There is not UI support available yet, you will have to write code!
To test if the plugin is installed and authenticated properly - type the following in the QGIS Python Console:


In [ ]:
>>> import ee
>>> print(ee.String('Hello World from EE!').getInfo())
Hello World from EE!

In [ ]:
A more advanced script may look like this:

<img src='./IMG/qgis_gee_2.png' alt='GEE console' align='center' width='75%'></img>
<br>

### Map

The plugin implements most of the (`Map.*`) functionality typically used in the **Code Editor**. Note, that no UI or Layers functionality is supported right now.

### Importing plugin

For most of the EE scripts, the following two imports must be included:

In [16]:
import ee
from ee_plugin import Map
import ee

ModuleNotFoundError: No module named 'ee_plugin'

<div class="alert alert-block alert-success" align="left">
<b><i>  "After that, the (`Map.*`) functions can be used in a similar way to the official EE Code Editor
The code above will query Earth Engine for an image and will add it as an XYZ tile layer to the QGIS Canvas."</i></b>
</div>

The code above will query Earth Engine for an image and will add it as an XYZ tile layer to the QGIS Canvas.

**Note:** that QGIS projects containing EE map layers can be also saved, in this case, the code required to connect to EE is stored in a QGIS project and is used to re-initialize these layers when the project is loaded. Currently, this works only if (`ee_plugin*`) is installed in the QGIS where these layers are loaded.

##	2. Mapping Burn area and fire severity on Google Earth Engine and QGIS using Python API example

This repository is developed as a Python example of the Google Earth Engine API, to monitor the burning area and map the severity of the fire Over Tunisia in the 2020 Season.

### Add QGIS Basemaps

* Open the Python console in QGIS and load the Python script (`Basemaps/qgis_basemaps.py*`) into the QGIS Python Editor.
* Click the Run script button on the Python Editor to execute the script. This will add many basesmaps as *XYZ tiles to QGIS*. 
* Select and double click any basemap under XYZ Ttiles to be added to QGIS Canvas. See the screenshot below.
Alternatively, you can install the QGIS QuickMapServices plugin. After installing the plugin, go to QGIS -- Web --QuickMapServices -- Settings -- More services -- Get contributed pack -- Save.

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/basemap.png' width='40%'/>
    <figcaption><i> Inserting a basemap on QGIS</i></figcaption>
</figure>
</div>

### Study area

This example focuses on Tunisia, for which we will use the LSIB line vector file and World Vector Shorelines (WVS) layer from the **National Geospatial-Intelligence Agency (NGA)**. 

This is a data set that has the regions in each country with a single characteristic. Furthermore, it excludes medium and small islands. The resulting simplified boundary lines rarely drift more than 100 meters from the detailed LSIB lines. 

This funtion add feature of theastudy area = (Tunisia):

In [ ]:
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
TUN = countries.filter(ee.Filter.eq('country_na', 'Tunisia'))

* Create an empty image into which to paint the features, cast to byte:

In [ ]:
empty = ee.Image().byte()

* Paint all the polygon edges with the same number and 'width', display.

In [ ]:
outline = empty.paint(**{
  'featureCollection': TUN,
  'color': 1,
  'width': 1.5
})

* Add the image to the map using definided color

In [ ]:
Map.addLayer(outline, {'palette': 'FF0000'}, 'Tunisia Limit')

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_3.png' width='75%'/>
    <figcaption><i> Openstreet Map on QGIS</i></figcaption>
</figure>
</div>

* set Map center using coordinates and zoom:

In [ ]:
Map.setCenter(9.346, 35.032, 6)

### Sentinel 2 imagen

This layer contain the Level-2 data produced by ESA can be found in the collection [COPERNICUS/S2_SR](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR).

Clouds can be mostly removed by using [COPERNICUS/S2_CLOUD_PROBABILITY](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY).

For more details on Sentinel-2 radiometric resolution, [see this page](https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/resolutions/radiometric).

* This funtion add a Sentinel 2 imagen:

In [ ]:
imagen = ee.ImageCollection('COPERNICUS/S2') \
  .filterBounds(TUN) \
  .filterDate('2020-04-15', '2020-08-19').median() \
  .divide(10000) \
  .select(
  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'],
  ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B08A', 'B11', 'B12']
  )
vis = {'bands': ['B12', 'B08', 'B04'], 'min': 0.05, 'max': 0.5}
Map.addLayer(imagen, vis, 'Sentinel 2 False color')
Map.setCenter(8.7107, 35.2061, 15)

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_4.png' width='75%'/>
    <figcaption><i> Sentinel 2A imagen</i></figcaption>
</figure>
</div>

### FIRMS from MODIS

This layer have the Fire Information for Resource Management System (FIRMS) dataset contains the LANCE fire detection product in rasterized form. The near real-time (NRT) active fire locations are processed by LANCE using the standard MODIS MOD14/MYD14 Fire and Thermal Anomalies product. Each active fire location represents the centroid of a 1km pixel that is flagged by the algorithm as containing one or more fires within the pixel. The data are rasterized as follows: for each FIRMS active fire point, a 1km bounding box (BB) is defined; pixels in the MODIS sinusoidal projection that intersect the FIRMS BB are identified; if multiple FIRMS BBs intersect the same pixel, the one with higher confidence is retained; in case of a tie, the brighter one is retained.

Additional information can be found [here](https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/about-firms).

* This function gets FIRMS from MODIS imagery for the date of 2020-04-15 to 2020-08-20:

In [ ]:
collection = ee.ImageCollection('FIRMS')\
  .filterBounds(TUN) \
  .select('T21')\
  .filterDate('2020-04-15', '2020-08-20')\

band_viz = {
  'min': 325,
  'max': 400,
  'palette': ['red', 'orange', 'yellow']
}

Map.addLayer(collection.mean(), band_viz, 'Hot Spots')

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_5.png' width='75%'/>
    <figcaption><i> Fire Information for Resource Management System (FIRMS)</i></figcaption>
</figure>
</div>

### Burn area products

Below we add the different burned area products available. the first is the **MODIS Fire_cci Burned Area** pixel product version 5.1 **(FireCCI51)** is a monthly global ~250m spatial resolution dataset containing information on burned area as well as ancillary data. It is based on surface reflectance in the Near Infrared **(NIR)** band from the **MODIS** instrument onboard the Terra satellite, as well as active fire information from the same sensor of the Terra and Aqua satellites.

* This funtion Visualize the burn area of FireCCI51 for the year 2019. Then we select (`BurnDate`) to estimated day of the year of the first detection of the burn:

In [ ]:
dataset = ee.ImageCollection('ESA/CCI/FireCCI/5_1')\
                  .filterDate('2019-01-01', '2019-12-30')
burnedArea = dataset.select('BurnDate')

##Use a circular palette to assign colors to date of first detection
baVis = {
  'min': 1,
  'max': 366,
  'palette': [
    'ff0000', 'fd4100', 'fb8200', 'f9c400', 'f2ff00', 'b6ff05',
    '7aff0a', '3eff0f', '02ff15', '00ff55', '00ff99', '00ffdd',
    '00ddff', '0098ff', '0052ff', '0210ff', '3a0dfb', '7209f6',
    'a905f1', 'e102ed', 'ff00cc', 'ff0089', 'ff0047', 'ff0004'
  ]
};
maxBA = burnedArea.max()
Map.addLayer(maxBA, baVis, 'Burned Area CCI Fire 2019')

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_6.png' width='75%'/>
    <figcaption><i> Burned Area CCI Fire 2019</i></figcaption>
</figure>
</div>

The next burn area product is the **MOD14A1 V6** dataset provides daily fire mask composites at **1km** resolution derived from the MODIS 4- and 11-micrometer radiances. The fire detection strategy is based on absolute detection of a fire (when the fire strength is sufficient to detect), and on detection relative to its background (to account for variability of the surface temperature and reflection by sunlight). The product distinguishes between fire, no fire and no observation.

For more information see the [MOD14 user guide](https://lpdaac.usgs.gov/sites/default/files/public/product_documentation/mod14_user_guide.pdf).

* This funtion Visualize the **MODIS/006/MOD14A1**:

In [ ]:
collection2 = ee.ImageCollection('MODIS/006/MOD14A1')\
.filterDate('2020-04-15', '2020-08-20')\
  
fireMaskVis = {
  'min': 0.0,
  'max': 6000.0,
  'bands': ['MaxFRP', 'FireMask', 'FireMask']
  }

Map.addLayer(collection2.mean(), fireMaskVis, 'MOD14A')

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_7.png' width='75%'/>
    <figcaption><i> MOD14A1 V6 MODIS Burned Area</i></figcaption>
</figure>
</div>

The next burn area product is the **MCD64A1 Version 6**, a Burned Area of Terra and Aqua combined data product is a monthly, global gridded 500m product containing per-pixel burned-area and quality information. The MCD64A1 burned-area mapping approach employs 500m MODIS Surface Reflectance imagery coupled with **1km** MODIS active fire observations. The algorithm uses a burn sensitive **vegetation index (VI)** to create dynamic thresholds that are applied to the composite data. The VI is derived from MODIS shortwave infrared atmospherically corrected surface reflectance bands 5 and 7 with a measure of temporal texture. The algorithm identifies the date of burn for the 500m grid cells within each individual **MODIS** tile. The date is encoded in a single data layer as the ordinal day of the calendar year on which the burn occurred, with values assigned to unburned land pixels and additional special values reserved for missing data and water grid cells.

* The next funtion Visualize the **MCD64A1.006** MODIS Burned Area Monthly Global 500m. Show the Burn day of year. Possible values: 0 (unburned), 1-366 (approximate Julian day of burning):

In [ ]:
#This funtion add a MCD64A1.006 MODIS Burned Area Monthly Global 500m
collection3 = ee.ImageCollection('MODIS/006/MCD64A1')\
.select('BurnDate')\
.filterDate('2020-01-01', '2020-06-01')\

fireMaskVis2 = {
  'min': 30.0,
  'max': 341.0,
  'palette': ['4e0400', '951003', 'c61503', 'ff1901']
  }

Map.addLayer(collection3.mean(), fireMaskVis2, 'MCD64A1')

### Burn severity mapping

Following is a mapping of the severity of fires using the normalized index of burned area **(NBR)**, which is applied in images of before and after a fire. To calculate its differential value or **(dNBR)**, the severity is derived, showing the area affected by the fire. The images used in this process come from the Sentinel-2 platform.

* This function gets pre NBR from Sentinel 2 imagery:

In [ ]:
imagen1 = ee.ImageCollection('COPERNICUS/S2') \
  .filterBounds(TUN) \
  .filterDate('2020-04-22', '2020-05-23').median() \
  .divide(10000) \
  .select(
  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'],
  ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B08A', 'B11', 'B12']
  )
vis2 = {'bands': ['B12', 'B08', 'B04'], 'min': 0.05, 'max': 0.5}
Map.addLayer(imagen1, vis2, 'Sentinel pre')

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_8.png' width='75%'/>
    <figcaption><i> Sentinel 2A Pre fire imagen</i></figcaption>
</figure>
</div>

* Next compute the NBR from the pre scene:

In [ ]:
nbr1 = imagen1.select('B08').subtract(imagen1.select('B11')) \
        .divide(imagen1.select('B08').add(imagen1.select('B11')))

nbrParams = {'palette': ['#d73027', '#f46d43', '#fdae61',
                          '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']}

Map.addLayer(nbr1, nbrParams, 'NBR PRE')

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_9.png' width='75%'/>
    <figcaption><i> NBR Pre fire imagen</i></figcaption>
</figure>
</div>

* This function gets the post NBR from Sentinel 2 imagery:

In [ ]:
imagen2 = ee.ImageCollection('COPERNICUS/S2') \
  .filterBounds(TUN) \
  .filterDate('2020-07-01', '2020-07-23').median() \
  .divide(10000) \
  .select(
  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'],
  ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B08A', 'B11', 'B12']
  )
vis3 = {'bands': ['B12', 'B08', 'B04'], 'min': 0.05, 'max': 0.5}
Map.addLayer(imagen2, vis3, 'Sentinel post')

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_10.png' width='75%'/>
    <figcaption><i> Sentinel 2 post fire imagen</i></figcaption>
</figure>
</div>

* Next Compute the NBR from the pre scene:

In [ ]:
nbr2 = imagen2.select('B08').subtract(imagen2.select('B11')) \
        .divide(imagen2.select('B08').add(imagen2.select('B11')))

nbrParams2 = {'palette': ['#d73027', '#f46d43', '#fdae61',
                          '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']}

Map.addLayer(nbr2, nbrParams2, 'NBR POST')

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_11.png' width='75%'/>
    <figcaption><i> NBR Post fire imagen</i></figcaption>
</figure>
</div>

* This funtion Compute the difference burn index dNBR image:

In [ ]:
dNBRDifference = (nbr2.subtract(nbr1)).multiply(1000)
difParams = {'min': -2000, 'max': 2000, 'palette': ['ffffff', 'FF0000', 'ff641b', 'ffaf38', 'fff70b', '0ae042', 'acbe4d', '7a8737' ]}

Map.addLayer(dNBRDifference, difParams, 'dNBR')

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_12.png' width='75%'/>
    <figcaption><i> dNBR imagen</i></figcaption>
</figure>
</div>

* To end and add the severity map with the different ranges, define and apply the diferents thresholds to the image:

In [ ]:
sld_intervals = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="intervals" extended="false" >' + \
      '<ColorMapEntry color="#ffffff" quantity="-500" label="-500"/>' + \
      '<ColorMapEntry color="#7a8737" quantity="-250" label="-250" />' + \
      '<ColorMapEntry color="#acbe4d" quantity="-100" label="-100" />' + \
      '<ColorMapEntry color="#0ae042" quantity="100" label="100" />' + \
      '<ColorMapEntry color="#fff70b" quantity="270" label="270" />' + \
      '<ColorMapEntry color="#ffaf38" quantity="440" label="440" />' + \
      '<ColorMapEntry color="#ff641b" quantity="660" label="660" />' + \
      '<ColorMapEntry color="#FF0000" quantity="2000" label="2000" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>';
# Add the image to the map using the color interval schemes.
Map.addLayer(dNBRDifference.sldStyle(sld_intervals), {}, 'dNBR intervals');

<br>
<div style='text-align:center;'>
<figure><img src='./IMG/qgis_gee_13.png' width='75%'/>
    <figcaption><i> dNBR severity classes imagen</i></figcaption>
</figure>
</div>